In [1]:
# 기본적인 분석 툴과 시각화 툴의 임포트
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# K-NN이 담긴 라이브러리 임포트
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
# 데이터 파일 불러오기
df=pd.read_csv("data/result_bs_1000.csv")

In [4]:
# 전처리에 필요한(테스트셋, 트레인셋)분리용 라이브러리 임포트
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
sns.set()

In [5]:
# 데이터 형태 미리보기
df.head(5)

,business_number,ked_code,avg_credit_payment_ratio,avg_credit_payment_period,credit_grade,suspension_date,avg_working_year,avg_employee_count,avg_rent_deposite,site_post_number,...,company_status,listing_YN,recent_date,accounts_receivable,asset,debt,net_income,non_current_liabilities,operating_profit,sales
0,3128113130,7325477,NaN,NaN,NaN,NaN,0.0,NaN,0.0,31186.0,...,1.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6212177145,2623657,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7397400074,2649241,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4102937030,9011039453,NaN,NaN,NaN,NaN,0.0,NaN,0.0,62044.0,...,1.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1353207178,2638158,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16528.0,...,1.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 데이터의 column 명 확인하기
list(df.columns.values)

['business_number',
 'ked_code',
 'avg_credit_payment_ratio',
 'avg_credit_payment_period',
 'credit_grade',
 'suspension_date',
 'avg_working_year',
 'avg_employee_count',
 'avg_rent_deposite',
 'site_post_number',
 'average_yearly_salary_total',
 'average_working_year_total',
 'per_share_par_value',
 'share_holders_count',
 'avg_company_size',
 'company_status',
 'listing_YN',
 'recent_date',
 'accounts_receivable',
 'asset',
 'debt',
 'net_income',
 'non_current_liabilities',
 'operating_profit',
 'sales']

In [7]:
# 파산 여부를 True/False로 저장하는 bankruptcy 열을 만들고
# 파산 일자의유무로 값을 지정
df['bankruptcy']=True
df['bankruptcy']=df['suspension_date'].notnull()

In [8]:
# 부채/자산 비율을 의미하는 debt/asset
# 매출채권/매출액을 의미하는 accounts_receivable/sales
# 위의 둘을 새로운 column으로 추가
df['debt/asset'] = df['debt']/df['asset']
df['accounts_receivable/sales'] = df['accounts_receivable']/df['sales']

In [9]:
# 쓸 수 있는 것과 없는 것을 알아보기 위해 결측치 현황을 보자.
print('결측치 현황\n')
print(df.isnull().sum())

결측치 현황

business_number                  0
ked_code                         0
avg_credit_payment_ratio       198
avg_credit_payment_period      198
credit_grade                    31
suspension_date                662
avg_working_year                38
avg_employee_count             203
avg_rent_deposite                0
site_post_number                10
average_yearly_salary_total    993
average_working_year_total     993
per_share_par_value            141
share_holders_count            141
avg_company_size                12
company_status                 447
listing_YN                       0
recent_date                    325
accounts_receivable            325
asset                          325
debt                           325
net_income                     325
non_current_liabilities        325
operating_profit               325
sales                          325
bankruptcy                       0
debt/asset                     325
accounts_receivable/sales      325
dtype: int64

In [10]:
# avg_company_size와 같이 값들이 무엇을 의미하는지 알 수 없는 변수는 제외

In [11]:
# 전체 변수의 리스트.
total_list = ['asset','debt', 'sales', 'accounts_receivable', 'avg_credit_payment_ratio', 'avg_credit_payment_period', 'avg_working_year', 'avg_employee_count', 'avg_rent_deposite', 'per_share_par_value', 'share_holders_count'
               , 'accounts_receivable/sales', 'debt/asset', 'net_income', 'non_current_liabilities', 'operating_profit', 'bankruptcy']
# 제거할 변수의 리스트. 기본적으로 아무것도 제거 안하는 상황
remove_list = []
# final_list는 total_list - remove_list
final_list = [item for item in total_list if item not in remove_list]

In [12]:
# 쓸 수 있는 것들과 파산 여부만 남긴 테이블을 만들자.
df_final = df[final_list]
# 결측치 제거
df_final = df_final.dropna(axis=0)
df_final.reset_index(drop=True, inplace=True)

In [13]:
# 우선 y가 될 데이터프레임을 새로 하나 만들자
y = pd.DataFrame()

# y값은 0 혹은 1로 해주어야 하므로 부도 여부를 0,1 로 나타내자
y['bankruptcy_binary'] = df_final['bankruptcy'].apply(lambda x: 1 if x==True else 0)

In [14]:
# bankruptcy 를 제외한 모든 변수를 X 로 둔다.
X = df_final.drop('bankruptcy',axis=1)

In [15]:
# 경고는 숨기자
import warnings
warnings.filterwarnings(action='ignore')

In [16]:
# 결과 데이터프레임을 만들자
result = pd.DataFrame(columns = ['var', 'knn_3', 'knn_4', 'knn_5'])

In [17]:
# 모든 변수가 추가된 경우를 basic case 라 할 때
# 이웃이 3,4,5 개인 경우 KNN 결과가 어떻게 나오는지 도출하자

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import random

# basic case. 모든 변수들이 다 포함되어 있음.
print('basic case: 모든 변수들이 다 포함된 경우')
result_row = []
result_row.append('basic')
for i in range(3, 6):
    print('* 이웃한 ' + str(i) + '개를 기준으로 분류한 경우')
   
    accuracy_sum = 0
    
    random_int = random.randint(0,9)
    cv = KFold(3, shuffle=True, random_state=random_int)
    for j, (idx_train, idx_test) in enumerate(cv.split(df_final)):
        X_train = X.iloc[idx_train]
        y_train = y.iloc[idx_train]
        X_test = X.iloc[idx_test]
        y_test = y.iloc[idx_test]
    
        # scale하는 코드
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        # knn train
        knn = KNeighborsClassifier(n_neighbors=i, metric='mahalanobis', metric_params={'V': np.cov(X_train.T)})
        knn.fit(X_train, y_train)
        # knn predict
        y_pred = knn.predict(X_test)

        # 정확도를 추산해보자
        accuracy =  accuracy_score(y_test, y_pred)
        accuracy_sum += accuracy
    avg_accuracy = round((accuracy_sum/3)*100,2)
    result_row.append(avg_accuracy)
    print('  random_state : ' + str(random_int) + ', 정확도는 ' + str(avg_accuracy) + '% 입니다.')
result = result.append({'var': result_row[0], 'knn_3': result_row[1], 'knn_4': result_row[2], 'knn_5': result_row[3]}, ignore_index=True)

basic case: 모든 변수들이 다 포함된 경우
* 이웃한 3개를 기준으로 분류한 경우
  random_state : 2, 정확도는 66.0% 입니다.
* 이웃한 4개를 기준으로 분류한 경우
  random_state : 4, 정확도는 62.0% 입니다.
* 이웃한 5개를 기준으로 분류한 경우
  random_state : 5, 정확도는 63.46% 입니다.


In [18]:
# 앞서 final list (현재는 아무것도 제외한 변수가 없으므로 total list 와 같다) 는 아래와 같다

# total_list = ['asset','debt', 'sales', 'accounts_receivable', 'avg_credit_payment_ratio', 'avg_credit_payment_period', 'avg_working_year', 'avg_employee_count', 'avg_rent_deposite', 'per_share_par_value', 'share_holders_count'
#              , 'accounts_receivable/sales', 'debt/asset', 'net_income', 'non_current_liabilities', 'operating_profit', 'bankruptcy']

# 측정 대상이 아닌 bankruptcy 를 제외하고 변수를 하나씩 제거하면서
# basic case 와 정확도를 비교해보자. 
# bankruptcy 를 제외하고 각 변수를 제거할 리스트를 new_total_list 라 하자

cnt = 0
# new_total_list 는 bankruptcy 를 제외한 변수명이 입력되어 있는 리스트
new_total_list = ['asset','debt', 'sales', 'accounts_receivable', 'avg_credit_payment_ratio', 'avg_credit_payment_period', 'avg_working_year', 'avg_employee_count', 'avg_rent_deposite', 'per_share_par_value', 'share_holders_count'
               , 'accounts_receivable/sales', 'debt/asset', 'net_income', 'non_current_liabilities', 'operating_profit']
for var in new_total_list:
    # 제거할 변수의 리스트. 기본적으로 아무것도 제거 안하는 상황
    remove_list = [var]
    # 이제 final_list는 total_list - remove_list임.
    final_list = [item for item in new_total_list if item not in remove_list]
    result_row = []
    result_row.append(str(var))
    print('제거된 변수: ' + '\''+ str(var) + '\'')
    for i in range(3, 6):
        print('* 이웃한 ' + str(i) + '개를 기준으로 분류한 경우')
       
        accuracy_sum = 0
       
        random_int = random.randint(0,9)
        cv = KFold(3, shuffle=True, random_state=random_int)
        for j, (idx_train, idx_test) in enumerate(cv.split(df_final)):
            X_train = X.iloc[idx_train]
            y_train = y.iloc[idx_train]
            X_test = X.iloc[idx_test]
            y_test = y.iloc[idx_test]

            # scale하는 코드
            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
            # knn train
            knn = KNeighborsClassifier(n_neighbors=i, metric='mahalanobis', metric_params={'V': np.cov(X_train.T)})
            knn.fit(X_train, y_train)
            # knn predict
            y_pred = knn.predict(X_test)

            # 정확도를 추산해보자
            accuracy =  accuracy_score(y_test, y_pred)
            accuracy_sum += accuracy
        avg_accuracy = round((accuracy_sum/3)*100,2)
        print('  random_state : ' + str(random_int) + ', 정확도는 ' + str(avg_accuracy) + '% 입니다.')
        result_row.append(avg_accuracy)
    print('-----------------------------------------')
    result = result.append({'var': result_row[0], 'knn_3': result_row[1], 'knn_4': result_row[2], 'knn_5': result_row[3]}, ignore_index=True)

제거된 변수: 'asset'
* 이웃한 3개를 기준으로 분류한 경우
  random_state : 3, 정확도는 62.18% 입니다.
* 이웃한 4개를 기준으로 분류한 경우
  random_state : 3, 정확도는 61.82% 입니다.
* 이웃한 5개를 기준으로 분류한 경우
  random_state : 5, 정확도는 63.46% 입니다.
-----------------------------------------
제거된 변수: 'debt'
* 이웃한 3개를 기준으로 분류한 경우
  random_state : 7, 정확도는 66.55% 입니다.
* 이웃한 4개를 기준으로 분류한 경우
  random_state : 0, 정확도는 60.55% 입니다.
* 이웃한 5개를 기준으로 분류한 경우
  random_state : 0, 정확도는 61.82% 입니다.
-----------------------------------------
제거된 변수: 'sales'
* 이웃한 3개를 기준으로 분류한 경우
  random_state : 7, 정확도는 66.55% 입니다.
* 이웃한 4개를 기준으로 분류한 경우
  random_state : 3, 정확도는 61.82% 입니다.
* 이웃한 5개를 기준으로 분류한 경우
  random_state : 1, 정확도는 64.18% 입니다.
-----------------------------------------
제거된 변수: 'accounts_receivable'
* 이웃한 3개를 기준으로 분류한 경우
  random_state : 6, 정확도는 64.73% 입니다.
* 이웃한 4개를 기준으로 분류한 경우
  random_state : 3, 정확도는 61.82% 입니다.
* 이웃한 5개를 기준으로 분류한 경우
  random_state : 9, 정확도는 66.72% 입니다.
-----------------------------------------
제거된 변수: 'avg_credit_payment_ratio'
* 이웃한 3개를 기준

In [19]:
# 이웃이 5개인 경우 이웃이 3/4개인 경우보다 
# 정확도가 basic case 와 차이가 많이남으로 이웃이 5개인 경우로 판단하자

# diff 열은 basic case 와의 정확도 차이를 의미한다
result['diff']=abs(result['knn_5']-result['knn_5'][0])

In [20]:
# over_mean 열은 basic case 와의 정확도 차이가 평균을 넘는가를 의미한다
result['over_mean']=(result['diff'] > result['diff'].mean())

In [21]:
result

,var,knn_3,knn_4,knn_5,diff,over_mean
0,basic,66.00,62.00,63.46,0.00,False
1,asset,62.18,61.82,63.46,0.00,False
2,debt,66.55,60.55,61.82,1.64,True
3,sales,66.55,61.82,64.18,0.72,False
4,accounts_receivable,64.73,61.82,66.72,3.26,True
5,avg_credit_payment_ratio,66.00,61.28,61.82,1.64,True
6,avg_credit_payment_period,64.73,62.35,64.18,0.72,False
7,avg_working_year,62.92,61.28,63.08,0.38,False
8,avg_employee_count,66.00,65.83,63.46,0.00,False
9,avg_rent_deposite,62.35,60.55,66.72,3.26,True
